In [28]:
%pip install -r requirements.txt -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
import os
import urllib.request
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

from langchain_openai import ChatOpenAI  
from langchain.chains import RetrievalQA  

In [30]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

pinecone_api_key = os.getenv('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = pinecone_api_key

In [31]:
df = pd.read_csv('/Users/mira/VScode/chatbot/data/my_profile.csv',header=None,names=['text','Category'])

# OpenAIEmbeddings インスタンスを作成
embeddings = OpenAIEmbeddings()

# 'text_column' の各セルをベクトル化してリストに保存
df['vectorized'] = df['text'].apply(lambda x: embeddings.embed_query(x))

# ベクトルリストをデータフレームに展開（各要素を別々のカラムに）
vectorized_df = pd.DataFrame(df['vectorized'].tolist(), index=df.index)

# 元のDataFrameとベクトル化したDataFrameを結合
output_df = pd.concat([df.drop(columns=['vectorized']), vectorized_df], axis=1)

In [32]:
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index("sample-db")
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'sample-db-rdebc4f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'sample-db',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [33]:
original_texts = output_df.iloc[:, 0]
original_category = output_df.iloc[:, 1]
vectorized_data_only = output_df.iloc[:, 2:]

In [34]:
for i in range(len(vectorized_data_only)):
    pinecone_index.upsert(
        vectors = [
            {
                'id': str(i+1),
                'values': vectorized_data_only.T[i],
                'metadata': {"text": original_texts[i], "Category": original_category[i]}
            }
        ]
    )

In [35]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'sample-db-rdebc4f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'sample-db',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [36]:
env="aws"
index_name = "sample-db"


embeddings = OpenAIEmbeddings()

#vectorstore = PineconeStore.from_documents(index_name, embeddings,"text")
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings,pinecone_api_key=pinecone_api_key)
#retriever = PineconeVectorStore(search_type="similarity", search_kwargs={"k": 5})

In [37]:

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.4,
)  
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  
#query = "趣味は？？"  
#qa.run(query)

q='研究室とかに所属してますか？'
result = qa.invoke({"query": q})
display("Answer:", result['result'])

'Answer:'

'はい、塩尻（斎藤）亜希研究室に所属しています。'

In [38]:
q='長期インターンシップで何をしていますか'
result = qa.invoke({"query": q})
display("Answer:", result['result'])

'Answer:'

'長期インターンシップでは、株式会社Zigexnにて機械学習エンジニアとしての業務に従事しています。また、業務委託として株式会社AlmondoでLLM開発のプロジェクトマネージャー（PjM）をしています。'

In [39]:
"""retrieval_qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model="gpt-3.5-turbo"),
        chain_type="refine",
        retriever=vectorstore.as_retriever(),
        return_source_documents=True
)
q='趣味はなんですか？'
result = retrieval_qa.invoke({"query": q})
display("Answer:", result['result'])"""

'retrieval_qa = RetrievalQA.from_chain_type(\n        llm=ChatOpenAI(model="gpt-3.5-turbo"),\n        chain_type="refine",\n        retriever=vectorstore.as_retriever(),\n        return_source_documents=True\n)\nq=\'趣味はなんですか？\'\nresult = retrieval_qa.invoke({"query": q})\ndisplay("Answer:", result[\'result\'])'

In [40]:
"""
# CSVファイルを読み込み、列名を指定
def load_data(filepath):
    return pd.read_csv(filepath, header=None, names=['text', 'Category'])

# テキストデータをベクトル化
def vectorize_text(df, embeddings):
    df['vectorized'] = df['text'].apply(lambda x: embeddings.embed_query(x))
    return df

# ベクトルデータをDataFrameに展開
def expand_vectors(df):
    vectorized_df = pd.DataFrame(df['vectorized'].tolist(), index=df.index)
    return pd.concat([df.drop(columns=['vectorized']), vectorized_df], axis=1)

# Pineconeにデータをアップロード
def upload_to_pinecone(index, output_df):
    original_texts = output_df.iloc[:, 0]
    original_category = output_df.iloc[:, 1]
    vectorized_data_only = output_df.iloc[:, 2:]

    for i in range(len(vectorized_data_only)):
        index.upsert(
            vectors=[
                {
                    'id': str(i + 1),
                    'values': vectorized_data_only.iloc[i].tolist(),
                    'metadata': {"text": original_texts[i], "Category": original_category[i]}
                }
            ]
        )

# PineconeVectorStoreの初期化
def initialize_vector_store(index_name, embeddings, pinecone_api_key):
    return PineconeVectorStore(index_name=index_name, embedding=embeddings, pinecone_api_key=pinecone_api_key)

# 質問応答システムのセットアップ
def setup_qa_system(vectorstore):
    llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0)
    return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

# メイン処理
def main():
    filepath = '/Users/ito_itsuki/Documents/python_env/chatbot/data/my_profile.csv'
    index_name = 'sample-db'
    
    df = load_data(filepath)
    
    # ベクトル化と展開
    embeddings = OpenAIEmbeddings()
    vectorized_df = expand_vectors(vectorize_text(df, embeddings))
    
    # Pineconeにアップロード
    pc = Pinecone(api_key=pinecone_api_key)
    pinecone_index = pc.Index(index_name)
    upload_to_pinecone(pinecone_index, vectorized_df)
    
    # 質問応答システムのセットアップと実行
    vectorstore = initialize_vector_store(index_name, embeddings, pinecone_api_key)
    qa = setup_qa_system(vectorstore)
    
    query = '研究室とかに所属してますか？'
    result = qa.invoke({"query": query})
    display("Answer:", result['result'])

if __name__ == "__main__":
    main()
"""

'\n# CSVファイルを読み込み、列名を指定\ndef load_data(filepath):\n    return pd.read_csv(filepath, header=None, names=[\'text\', \'Category\'])\n\n# テキストデータをベクトル化\ndef vectorize_text(df, embeddings):\n    df[\'vectorized\'] = df[\'text\'].apply(lambda x: embeddings.embed_query(x))\n    return df\n\n# ベクトルデータをDataFrameに展開\ndef expand_vectors(df):\n    vectorized_df = pd.DataFrame(df[\'vectorized\'].tolist(), index=df.index)\n    return pd.concat([df.drop(columns=[\'vectorized\']), vectorized_df], axis=1)\n\n# Pineconeにデータをアップロード\ndef upload_to_pinecone(index, output_df):\n    original_texts = output_df.iloc[:, 0]\n    original_category = output_df.iloc[:, 1]\n    vectorized_data_only = output_df.iloc[:, 2:]\n\n    for i in range(len(vectorized_data_only)):\n        index.upsert(\n            vectors=[\n                {\n                    \'id\': str(i + 1),\n                    \'values\': vectorized_data_only.iloc[i].tolist(),\n                    \'metadata\': {"text": original_texts[i], "Categor